# Curriculum 2: Designing Infrastructure and Managing Migration

## Lesson 5: Automation

### Concept 1: Intro
- ARM template deployment
- ARM template modification
- Desired state configuration
- Azure polices

### Concept 3: Deploy using ARM Template

Azure portal (deploy a cutom template) -> JSON template -> deploy -> VM !

`Parameters.json` -> `Template.json` -> Azure VM:
- schema
- contentVersion
- Parameters
- Variables
- Functions
- Resources
- outputs

### Concept 5: Exercise, Deploy a Storage Account using an ARM Template

Создаем ARM шаблон:
- Azure portal -> Storage accounts -> create:
    - Resource group
    - Storage account name: `storagefullmoon1`
    - Redundancy: GRS
    - Review + create -> download a template for automation -> download
- Unzip `template.zip` -> `parameters.json` -> `storageAccountName` to `null`:
```json
    "storageAccountName": {
        "value": null
    },
```

Деплой ARM шаблон:
- Azure portal -> Deploy a custom template -> Build your own template in editor -> Load file `template.json` -> save
- Edit parameters -> Load file `parameters.json` -> Заполняются все поля кроме Resource group и Storage Account Name (который null)
    - Resource group
    - Storage Account name: `storagefullmoon2`
- Go to resources и проверяем, что Storage account создался норм

### Concept 11: Exercise, Use Desired State Configuration to keep our Web Servers configured properly

С помощью Desired State Configuration мы можем автоматизировать процесс установки ролей для серверов, развернутых в качестве веб-серверов.

Создаем VM:
- Azure portal -> deploy a custom template -> create windows VM:
    - Resourece group
    - admin: `vmadmin` + pass
    - Review + create -> create

Создаем automation account и настраиваем DSC:
- Azure portal -> automation accounts -> create:
    - Resource group
    - Name: `automatewebservers`
    - Review + create -> create
- Go to resource -> State configuration (DSC) -> configurations (вкладка) -> add:
    - Local create file `iis.ps1` and load it by name `IISInstall3`:

        ```PowerShell
        Configuration IISInstall3
        {
          node "localhost" {
            WindowsFeature WebServer {
              Ensure ="Present"
              Name = "Web-Server"
            }
          }
        }
        ```

    - Deep in IISInstall3 -> compile -> Yes -> wait for few minutes -> status Completed
- Nodes (вкладка) -> add:
    - check our VM -> connect
        - Node configuration name: `IISInstall3.localhost`
        - Пропустил Configuration mode : `ApplyandAutoCorrect`
        - Allow module override: X
        - Reboot node if needed: X
        - Ok -> wait for few minutes -> status Compliant
        
Тестируем DSC:
- Azure portal -> VMs -> our `simple-vm` -> connect RDP (piblic id + 3389) + `vmadmin` + pass
- Server manager -> Manage (верхний пр угол) -> Remove roles and features:
    - Next, next -> server roles -> uncheck IIS -> next -> remove
- Restart VM (in Azure) -> reconnect VM RDP after few minutes
- После перезагрузки можем проверить, что в Add role and features пункт IIS снова включен

PS: я лично этого не заметил. Статус моего state configuration был Pending (в ожидании).  
Оказывается, Всё из-за того, что в момент регистрации VM я не выбрал Configuration mode : `ApplyandAutoCorrect`.  
Если перейти в `automatewebservers`, то можем увидеть, что появилась запись No compliant. Вроде бы как после запуска скрипта статус должен измениться на Compliant.


### Concept 14: Exercise, Azure polices

Просмотр  Azure police для предотвращения развертывания VM разного размера.

- Azure cloud -> policy -> change new Scope -> choose Resource group `staging` -> select -> 
- click view all 
- check Allowed VM size SKUs -> страница подтверждает 100% согласие ресурсов

Протестируем создание VM другого размера:
- Azure portal -> VM:
    - Resource group: `staging`
    - Unnable to pick size `Standard E2s_v3`: Blocked by Policy
    - Allow size `Standard DS1_v2` size
    
PS: политики действуют на выбор любых значений. Грубо говоря это административные фильтры.

### Concept 16: Blueprint

**Blueprints** — сервис Azure Cloud, позволяющий в декларативном виде определять повторяемый набор ресурсов Azure Cloud, соответствующий стандартам и требованиям организации.

Blueprints — подобие группировки templates.

> Замечу что blueprint assignment может довольно интенсивно расходовать лимит на API calls, который не очень велик. Поэтому при небольших изменениях конфигурации лучше применять что-то более fine-grained. Мы blueprints используем когда развёртываем новую ресурсную группу с нуля, это действительно очень удобно.

[source](https://habr.com/ru/company/dataart/blog/540956/#comment_22638210)

Артефакты:
- Roles
- Politics
- ARM templates
- Resource groups

### Concept 18: Exercise, view a Blueprint

Знакомимся с Blueprints
- Azure portal -> Blueprints -> Blueprint definitions -> check `Presetpolices`
- Артефакты хранятся в виде дерева

Тестируем Blueprint:
- Azure portal -> Storage accounts -> create
    - Resource group: `Production`
    - Name: any
    - Create
- Go to resource -> видим, что Tag проставился автоматически
- Если попробовать создать VM для `Production` resource group, то столкнемся с политикой, которая не позволяет это сделать.

### Concept 20: Edge cases

Типы ошибок ARM template:
- Validation
- Deployment errors